# जेनेरेटिभ नेटवर्कहरू

Recurrent Neural Networks (RNNs) र तिनका गेटेड सेल भेरियन्टहरू जस्तै Long Short Term Memory Cells (LSTMs) र Gated Recurrent Units (GRUs) ले भाषा मोडलिङको लागि एउटा मेकानिज्म प्रदान गरे, अर्थात् यीले शब्दहरूको क्रम सिक्न सक्छन् र अनुक्रममा आउने अर्को शब्दको भविष्यवाणी गर्न सक्छन्। यसले हामीलाई RNNs लाई **जेनेरेटिभ कार्यहरू** जस्तै साधारण पाठ उत्पादन, मेसिन अनुवाद, र यहाँसम्म कि छविको क्याप्शनिङको लागि प्रयोग गर्न अनुमति दिन्छ।

पछिल्लो युनिटमा हामीले छलफल गरेको RNN आर्किटेक्चरमा, प्रत्येक RNN युनिटले अर्को लुकेको अवस्था (hidden state) उत्पादन गर्थ्यो। तर, हामी प्रत्येक पुनरावर्ती युनिटमा अर्को आउटपुट पनि थप्न सक्छौं, जसले हामीलाई एउटा **अनुक्रम** (जसको लम्बाइ मूल अनुक्रमसँग बराबर हुन्छ) उत्पादन गर्न अनुमति दिन्छ। अझै, हामी त्यस्ता RNN युनिटहरू प्रयोग गर्न सक्छौं जसले प्रत्येक चरणमा इनपुट स्वीकार्दैनन्, तर केवल कुनै सुरुवाती अवस्था भेक्टर लिन्छन्, र त्यसपछि आउटपुटहरूको अनुक्रम उत्पादन गर्छन्।

यस नोटबुकमा, हामी सरल जेनेरेटिभ मोडेलहरूमा केन्द्रित हुनेछौं जसले हामीलाई पाठ उत्पादन गर्न मद्दत गर्छ। सरलताका लागि, आउनुहोस् **क्यारेक्टर-स्तरको नेटवर्क** निर्माण गरौं, जसले अक्षर-प्रति-अक्षर पाठ उत्पादन गर्छ। प्रशिक्षणको क्रममा, हामीले केही पाठ कर्पस लिनुपर्छ, र यसलाई अक्षर अनुक्रमहरूमा विभाजन गर्नुपर्छ।


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## वर्णको शब्दकोश निर्माण

वर्ण-स्तरको जनरेटिभ नेटवर्क निर्माण गर्न, हामीलाई पाठलाई शब्दहरूमा होइन, व्यक्तिगत वर्णहरूमा विभाजन गर्न आवश्यक छ। यो फरक टोकनाइजर परिभाषित गरेर गर्न सकिन्छ:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


हामीले हाम्रो डाटासेटबाट पाठलाई कसरी एन्कोड गर्न सक्छौं भन्ने उदाहरण हेरौं:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## जेनेरेटिभ RNN प्रशिक्षण

हामी RNN लाई पाठ उत्पन्न गर्न प्रशिक्षण दिने तरिका निम्नानुसार हुनेछ। प्रत्येक चरणमा, हामी `nchars` लम्बाइको अक्षरहरूको श्रृंखला लिनेछौं, र प्रत्येक इनपुट अक्षरको लागि नेटवर्कलाई अर्को आउटपुट अक्षर उत्पन्न गर्न अनुरोध गर्नेछौं:

![RNN ले 'HELLO' शब्द उत्पन्न गरिरहेको उदाहरण देखाउने छवि।](../../../../../translated_images/ne/rnn-generate.56c54afb52f9781d.webp)

वास्तविक परिदृश्यमा निर्भर गर्दै, हामीले केही विशेष अक्षरहरू पनि समावेश गर्न चाहन सक्छौं, जस्तै *end-of-sequence* `<eos>`। हाम्रो अवस्थामा, हामी केवल नेटवर्कलाई अन्तहीन पाठ उत्पन्न गर्न प्रशिक्षण दिन चाहन्छौं, त्यसैले हामी प्रत्येक श्रृंखलाको आकारलाई `nchars` टोकनहरूको बराबरमा स्थिर गर्नेछौं। तदनुसार, प्रत्येक प्रशिक्षण उदाहरण `nchars` इनपुटहरू र `nchars` आउटपुटहरू (जसले इनपुट श्रृंखलालाई एक प्रतीक बायाँ सिफ्ट गरेको हुन्छ) बाट बनेको हुनेछ। मिनिब्याचमा यस्ता धेरै श्रृंखलाहरू हुनेछन्।

हामी मिनिब्याचहरू उत्पन्न गर्ने तरिका यस्तो हुनेछ: प्रत्येक समाचार पाठको लम्बाइ `l` लिनेछौं, र त्यसबाट सबै सम्भावित इनपुट-आउटपुट संयोजनहरू उत्पन्न गर्नेछौं (त्यहाँ `l-nchars` यस्ता संयोजनहरू हुनेछन्)। यी संयोजनहरूले एउटा मिनिब्याच बनाउनेछन्, र प्रत्येक प्रशिक्षण चरणमा मिनिब्याचहरूको आकार फरक हुनेछ।


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

अब हामी जेनरेटर नेटवर्क परिभाषित गरौं। यो कुनै पनि पुनरावर्ती सेलमा आधारित हुन सक्छ जुन हामीले अघिल्लो युनिटमा छलफल गरेका थियौं (साधारण, LSTM वा GRU)। हाम्रो उदाहरणमा, हामी LSTM प्रयोग गर्नेछौं।

किनभने नेटवर्कले अक्षरहरूलाई इनपुटको रूपमा लिन्छ, र शब्दकोशको आकार धेरै सानो छ, हामीलाई embedding लेयरको आवश्यकता पर्दैन, one-hot-encoded इनपुट सिधै LSTM सेलमा जान सक्छ। तर, किनभने हामी अक्षरहरूको संख्या इनपुटको रूपमा पास गर्छौं, हामीले तिनीहरूलाई LSTM मा पास गर्नु अघि one-hot-encode गर्न आवश्यक छ। यो `forward` पासको क्रममा `one_hot` फंक्शन कल गरेर गरिन्छ। आउटपुट इन्कोडर भनेको एउटा linear लेयर हुनेछ जसले लुकेको अवस्थालाई one-hot-encoded आउटपुटमा रूपान्तरण गर्नेछ।


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

प्रशिक्षणको क्रममा, हामीले उत्पन्न गरिएको पाठलाई नमूना बनाउन सक्षम हुन चाहन्छौं। त्यसका लागि, हामी `generate` नामक एक कार्य परिभाषित गर्नेछौं जसले प्रारम्भिक स्ट्रिङ `start` बाट सुरु गर्दै, लम्बाइ `size` को आउटपुट स्ट्रिङ उत्पादन गर्नेछ।

यसले काम गर्ने तरिका निम्नानुसार छ। पहिलो, हामी सम्पूर्ण `start` स्ट्रिङलाई नेटवर्क मार्फत पास गर्नेछौं, र आउटपुट अवस्था `s` र अर्को भविष्यवाणी गरिएको अक्षर `out` लिनेछौं। किनभने `out` एक-हट एन्कोड गरिएको छ, हामी `argmax` लिन्छौं ताकि शब्दकोशमा अक्षर `nc` को सूचकांक प्राप्त गर्न सकियोस्, र `itos` प्रयोग गरेर वास्तविक अक्षर पत्ता लगाई अक्षरहरूको परिणामस्वरूप सूची `chars` मा थप्न सकियोस्। यो प्रक्रिया, एक अक्षर उत्पन्न गर्ने, `size` पटक दोहोर्याइन्छ ताकि आवश्यक संख्याको अक्षरहरू उत्पन्न गर्न सकियोस्।


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

अब प्रशिक्षण गरौं! प्रशिक्षण लूप हाम्रा अघिल्ला उदाहरणहरूजस्तै छ, तर यसपटक हामी सटीकता प्रिन्ट गर्ने सट्टा प्रत्येक १००० इपोक्समा नमूना गरिएको उत्पन्न पाठ प्रिन्ट गर्छौं।

विशेष ध्यान हामीले कसरी हानि गणना गर्छौं भन्ने कुरामा दिनुपर्छ। हामीले एक-हट-एन्कोड गरिएको आउटपुट `out` र अपेक्षित पाठ `text_out` (जसले क्यारेक्टर इन्डेक्सहरूको सूची प्रतिनिधित्व गर्छ) दिइएको अवस्थामा हानि गणना गर्नुपर्छ। भाग्यवश, `cross_entropy` फंक्शनले पहिलो तर्कको रूपमा अनन्यस्त (unnormalized) नेटवर्क आउटपुट र दोस्रो तर्कको रूपमा वर्ग संख्या (class number) अपेक्षा गर्छ, जुन ठ्याक्कै हामीसँग छ। यसले मिनिब्याच साइजमा स्वचालित औसत गणना पनि गर्छ।

हामीले `samples_to_train` नमूनाहरूद्वारा प्रशिक्षण सीमित गर्छौं, ताकि धेरै समय कुर्नु नपरोस्। हामी तपाईंलाई लामो समयसम्म प्रशिक्षण गर्ने प्रयास गर्न प्रोत्साहित गर्छौं, सम्भवतः केही इपोक्ससम्म (जसको लागि तपाईंले यो कोड वरिपरि अर्को लूप सिर्जना गर्नुपर्नेछ)।


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

यो उदाहरणले पहिले नै राम्रो पाठ उत्पादन गर्छ, तर यसलाई अझ सुधार गर्न केही तरिकाहरू छन्:

* **मिनिब्याच उत्पादनलाई अझ राम्रो बनाउनुहोस्।** हामीले प्रशिक्षणका लागि डाटा तयार गर्दा एउटा नमूनाबाट एउटा मिनिब्याच उत्पादन गर्‍यौं। यो आदर्श होइन, किनभने मिनिब्याचहरू सबै फरक आकारका हुन्छन्, र केही अवस्थामा त उत्पादन नै गर्न सकिँदैन, किनभने पाठ `nchars` भन्दा सानो हुन्छ। साथै, साना मिनिब्याचहरूले GPU लाई पर्याप्त रूपमा लोड गर्दैनन्। सबै नमूनाहरूबाट एउटा ठूलो पाठको टुक्रा लिनु, त्यसपछि सबै इनपुट-आउटपुट जोडीहरू उत्पादन गर्नु, तिनीहरूलाई शफल गर्नु, र समान आकारका मिनिब्याचहरू उत्पादन गर्नु अझ बुद्धिमानी हुनेछ।

* **मल्टिलेयर LSTM।** 2 वा 3 तहका LSTM सेलहरू प्रयास गर्नु उपयुक्त हुन्छ। जस्तै हामीले अघिल्लो युनिटमा उल्लेख गर्‍यौं, LSTM को प्रत्येक तहले पाठबाट निश्चित ढाँचाहरू निकाल्छ। क्यारेक्टर-स्तरको जेनेरेटरको अवस्थामा, हामी अपेक्षा गर्न सक्छौं कि तल्लो LSTM तहले अक्षरहरूको समूह (syllables) निकाल्न जिम्मेवार हुनेछ, र माथिल्लो तहहरूले शब्द र शब्द संयोजनहरू निकाल्नेछन्। यो LSTM कन्स्ट्रक्टरमा तहहरूको संख्या (number-of-layers) को प्यारामिटर पास गरेर सजिलै कार्यान्वयन गर्न सकिन्छ।

* तपाईंले **GRU युनिटहरू** प्रयोग गरेर पनि परीक्षण गर्न चाहन सक्नुहुन्छ, र कुन राम्रो प्रदर्शन गर्छ भनेर हेर्न सक्नुहुन्छ। साथै, **विभिन्न लुकेको तहको आकार (hidden layer sizes)** सँग पनि परीक्षण गर्न सक्नुहुन्छ। धेरै ठूलो लुकेको तहले ओभरफिटिंगको कारण दिन सक्छ (जस्तै, नेटवर्कले ठ्याक्कै पाठ सिक्नेछ), र सानो आकारले राम्रो नतिजा उत्पादन नगर्न सक्छ।


## नरम पाठ उत्पादन र तापक्रम

`generate` को अघिल्लो परिभाषामा, हामी सधैं उच्चतम सम्भावना भएको अक्षरलाई उत्पन्न पाठको अर्को अक्षरको रूपमा लिइरहेका थियौं। यसले गर्दा पाठ प्रायः बारम्बार उही अक्षर अनुक्रमहरूमा "चक्र" हुने गर्थ्यो, जस्तै यो उदाहरणमा:
```
today of the second the company and a second the company ...
```

तर, यदि हामी अर्को अक्षरको लागि सम्भावना वितरणलाई हेर्‍यौं भने, केही उच्चतम सम्भावनाहरूको बीचको भिन्नता धेरै ठूलो नहुन सक्छ, जस्तै एउटा अक्षरको सम्भावना ०.२ हुन सक्छ, अर्कोको ०.१९, आदि। उदाहरणका लागि, '*play*' अनुक्रममा अर्को अक्षर खोज्दा, अर्को अक्षर समान रूपमा खाली ठाउँ वा **e** (जस्तै *player* शब्दमा) हुन सक्छ।

यसले हामीलाई यो निष्कर्षमा पुर्‍याउँछ कि सधैं उच्च सम्भावना भएको अक्षर चयन गर्नु "न्यायोचित" हुँदैन, किनभने दोस्रो उच्चतम चयन गर्दा पनि अर्थपूर्ण पाठमा पुग्न सकिन्छ। यो बढी बुद्धिमानी हुन्छ कि नेटवर्कको आउटपुटले दिएको सम्भावना वितरणबाट अक्षरहरू **नमूना** गरियोस्।

यो नमूना `multinomial` नामक कार्य प्रयोग गरेर गर्न सकिन्छ, जसले **multinomial distribution** लागू गर्छ। यो **नरम** पाठ उत्पादन कार्य निम्न रूपमा परिभाषित गरिएको छ:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

हामीले **तापक्रम** नामक अर्को प्यारामिटर प्रस्तुत गरेका छौं, जसले उच्च सम्भावनामा कत्तिको कडाइका साथ टाँसिनुपर्छ भनेर संकेत गर्न प्रयोग गरिन्छ। यदि तापक्रम १.० छ भने, हामी निष्पक्ष बहुपद नमूना लिन्छौं, र जब तापक्रम अनन्ततिर जान्छ - सबै सम्भावनाहरू समान हुन्छन्, र हामी अर्को अक्षर अनियमित रूपमा चयन गर्छौं। तलको उदाहरणमा, हामीले देख्न सक्छौं कि तापक्रम धेरै बढाउँदा पाठ अर्थहीन बन्छ, र जब यो ० नजिक हुन्छ, यो "चक्रित" कडा-उत्पन्न पाठ जस्तै देखिन्छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
